In [1]:
import pandas as pd
import numpy as np
from keras.preprocessing.sequence import pad_sequences
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Lambda
from keras.layers import LSTM, Dropout
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
import tensorflow as tf

Using TensorFlow backend.


In [22]:
df = pd.read_csv('../../data/scheme-progs.csv')
df = df.sample(frac=1).reset_index(drop=True)
df = df[df.output != 'E']
df.output = pd.to_numeric(df.output)

In [28]:
37389*.8

29911.2

In [31]:
train_x = df.iloc[:30000,0:1].values[:,0]
train_y = df.iloc[:30000,1].values.reshape((30000, 1))
test_x = df.iloc[30000:,0:1].values[:,0]
test_y = df.iloc[30000:,1].values.reshape((7389, 1))

print(train_x.shape, train_y.shape, test_x.shape, test_y.shape)

(30000,) (30000, 1) (7389,) (7389, 1)


In [32]:
SYMBOLS = ['(', ')', ' ', 'x', 'y', '1', '2', '0', '-', 'λ', '+', '-', '*', '/']
N_SYMBOLS = len(SYMBOLS)
TOKENS = dict((c, i) for i, c in enumerate(SYMBOLS))
MAX_LEN = max(list(map(len, (list(train_x)+list(test_x)))))

def tokenize_string(s):
    ret = np.zeros((MAX_LEN, N_SYMBOLS), dtype=bool)
    for i, char in enumerate(s):
        ret[i, TOKENS[char]] = 1
    return ret

def tokenize(a):
#     ret = np.zeros(len(a), MAX_LEN, N_SYMBOLS)
#     sentences = 
#     for i, sentence in enumerate(s):
#         for t, char in enumerate(sentence):
    return np.array(list(map(lambda s: tokenize_string(s), list(a))))

In [33]:
train_x = tokenize(train_x)
test_x = tokenize(test_x)

print(train_x.shape, train_y.shape, test_x.shape, test_y.shape)

(30000, 35, 14) (30000, 1) (7389, 35, 14) (7389, 1)


In [34]:
model = Sequential()
model.add(LSTM(128, input_shape=(MAX_LEN,N_SYMBOLS)))
model.add(Dropout(0.2))
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dense(1))

model.compile(loss='mean_squared_error', optimizer='Adam', metrics=['accuracy'])
model.fit(train_x,
          train_y,
          batch_size=500,
          epochs=10,
          validation_split=0.3,
          callbacks=[EarlyStopping(patience=2)])

Train on 21000 samples, validate on 9000 samples
Epoch 1/10
21000/21000 [==============================] - 23s - loss: 3.2520 - acc: 0.2191 - val_loss: 2.9914 - val_acc: 0.2230
Epoch 2/10
21000/21000 [==============================] - 21s - loss: 2.7939 - acc: 0.2346 - val_loss: 2.6749 - val_acc: 0.2424
Epoch 3/10
21000/21000 [==============================] - 21s - loss: 2.6706 - acc: 0.2379 - val_loss: 2.6031 - val_acc: 0.2421
Epoch 4/10
21000/21000 [==============================] - 22s - loss: 2.5718 - acc: 0.2412 - val_loss: 2.5679 - val_acc: 0.2459
Epoch 5/10
21000/21000 [==============================] - 21s - loss: 2.5509 - acc: 0.2438 - val_loss: 2.5134 - val_acc: 0.2500
Epoch 6/10
21000/21000 [==============================] - 21s - loss: 2.5044 - acc: 0.2407 - val_loss: 2.4738 - val_acc: 0.2493
Epoch 7/10
21000/21000 [==============================] - 21s - loss: 2.4352 - acc: 0.2471 - val_loss: 2.3694 - val_acc: 0.2589
Epoch 8/10
21000/21000 [==============================]

In [35]:
model.evaluate(x=test_x, y=test_y, batch_size=500)

7389/7389 [==============================] - 2s     


[2.1470462619927599, 0.27324401323770153]

In [62]:
tests = tokenize(np.array(['((λ (x) (x 1)) (λ (y) (* 2 y)))', '((λ (x) (+ x 1)) 2)', '(+ 1 1)', '(/ 1 2)', '(* 2 2)', '(- 0 1)', '(+ 1 (+ 1 1))']))

In [63]:
model.predict(tests)

array([[ 1.60488272],
       [ 2.6646111 ],
       [ 2.53276873],
       [ 0.9617548 ],
       [ 3.51821947],
       [-0.34459147],
       [ 3.12763119]], dtype=float32)